In [3]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt




ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [16]:
# Choose dataset 1 or 2:
dataset = 2  # 1 or 2

if dataset == 1:
    data_folder = "1_train-val_1min_aalesund_from_start/"
    yaml_file_all = "data_1.yaml"
    yaml_file_ball = "ball_1.yaml"
    yaml_file_players = "player_1.yaml"

else:
    data_folder = "2_train-val_1min_after_goal/"
    yaml_file_all = "data_2.yaml"
    yaml_file_ball = "ball_2.yaml"
    yaml_file_players = "player_2.yaml"


In [9]:
# Not working?

# Generate shuffled txt and img files for training and validation, 80% training and 20% validation

from shuffler import generate_shuffled_txt_img_files_for_train_val

base_path = f"data_yolov8/{data_folder}/"  
text_file_path = f"/datasets/tdt4265/other/rbk/{data_folder}/gt/gt.txt"  
all_images_path = f"/datasets/tdt4265/other/rbk/{data_folder}/img1"
img_width = 1920  
img_height = 1080  

generate_shuffled_txt_img_files_for_train_val(text_file_path, img_width, img_height, base_path, all_images_path)


In [ ]:
# Generate test txt and img files

from test_dataset import generate_txt_img_files_for_test

base_path = "data_yolov8/3_test_1min_hamkam_from_start/"  
text_file_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/gt/gt.txt"  
all_images_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/img1/"

generate_txt_img_files_for_test(text_file_path, img_width, img_height, base_path, all_images_path)

In [3]:
# Last inn sist trente modell
# Gjør det ikke bedre virker det som

def find_latest_model_with_best(base_path):
    detect_path = os.path.join(base_path, 'runs', 'detect')
    training_sessions = [os.path.join(detect_path, d) for d in os.listdir(detect_path) if os.path.isdir(os.path.join(detect_path, d))]
    
    # Liste for å holde stiene til alle 'best.pt'-filer
    best_model_paths = []

    for session in training_sessions:
        best_model_path = os.path.join(session, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            best_model_paths.append((best_model_path, os.path.getmtime(best_model_path)))

    if not best_model_paths:
        print("Ingen 'best.pt' fil funnet i noen av treningsøktene.")
        return None
    
    # Sorter basert på modifiseringstid og returner den nyeste
    best_model_paths.sort(key=lambda x: x[1], reverse=True)
    return best_model_paths[0][0]


base_path = '/work/mbergst/TDT4265_Project' 
latest_model_path_with_best = find_latest_model_with_best(base_path)
if latest_model_path_with_best:
    print(f"Siste 'best.pt' modell funnet på: {latest_model_path_with_best}")



model = YOLO(latest_model_path_with_best)


Siste 'best.pt' modell funnet på: /work/mbergst/TDT4265_Project/runs/detect/train162/weights/best.pt


In [14]:
# Object Detection

# Tren modellen på datasettet
results = model.train(data=yaml_file_all, epochs=2, batch=8, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect')
#results = model.train(data=yaml_file_ball, epochs=6, batch=8, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect')
#results = model.train(data=yaml_file_players, epochs=6, batch=8, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect')




New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
engine/trainer: task=detect, mode=train, model=/work/mbergst/TDT4265_Project/runs/detect/train162/weights/best.pt, data=data_1.yaml, epochs=2, time=None, patience=100, batch=8, imgsz=(1920, 1080), save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/mbergst/TDT4265_Project/runs/detect, name=train1722232, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=

train: Scanning /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/train.cache... 1441 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1441/1441 [00:00<?, ?it/s]
val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]


Plotting labels to /work/mbergst/TDT4265_Project/runs/detect/train1722232/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1920 train, 1920 val
Using 8 dataloader workers
Logging results to /work/mbergst/TDT4265_Project/runs/detect/train1722232
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2        15G     0.8802      1.355     0.9404         72       1920: 100%|██████████| 181/181 [00:28<00:00,  6.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00,  7.71it/s]

                   all        361       8361      0.987      0.885      0.918       0.71


...

        2/2      13.9G     0.5266     0.4396     0.8004         63       1920: 100%|██████████| 181/181 [00:28<00:00,  6.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00, 10.42it/s]

                   all        361       8361      0.978      0.897      0.918       0.72



2 epochs completed in 0.018 hours.
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/detect/train1722232/weights/last.pt, 6.5MB
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/detect/train1722232/weights/best.pt, 6.5MB

Validating /work/mbergst/TDT4265_Project/runs/detect/train1722232/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:04<00:00,  4.82it/s]


                   all        361       8361      0.978      0.897      0.918       0.72
                  ball        361        241       0.97      0.797      0.841      0.552
                player        361       8120      0.987      0.998      0.995      0.888
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to /work/mbergst/TDT4265_Project/runs/detect/train1722232


In [7]:
# Evaluér modellens prestasjon på valideringssettet
val_results = model.val()


Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:03<00:00, 12.45it/s]


                   all        361       8361      0.968      0.842      0.879      0.699
                  ball        361        241      0.948      0.685      0.764      0.498
                player        361       8120      0.989      0.998      0.995        0.9
Speed: 0.4ms preprocess, 3.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /work/mbergst/TDT4265_Project/runs/detect/train142


In [6]:
# Test modellen på testsettet
test_results = model.val(data='test.yaml')

Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/3_test_1min_hamkam_from_start/labels/test.cache... 1802 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1802/1802 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 226/226 [00:12<00:00, 18.35it/s]


KeyError: 1

In [11]:
# Object tracking
all_frames = f"/work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/1_train-val_1min_aalesund_from_start/images/all_frames"

#all_frames = f"/work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames"

frame_paths = sorted([os.path.join(all_frames, f) for f in os.listdir(all_frames) if f.endswith('.jpg')])





In [15]:
# Iterere over hver frame og bruk track metoden
for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_width=1):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene


image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 1 player, 3.8ms
Speed: 2.3ms preprocess, 3.8ms inference, 0.8ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/1_train-val_1min_aalesund_from_start/images/all_frames/000002.jpg: 1088x1920 22 players, 3.0ms
Speed: 2.6ms preprocess, 3.0ms inference, 0.9ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/1_train-val_1min_aalesund_from_start/images/all_frames/000003.jpg: 1088x1920 22 players, 3.2ms
Speed: 2.9ms preprocess, 3.2ms inference, 0.6ms postprocess per image at shape (1, 3, 1088, 1920)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/1_train-val_1min_aalesund_from_start/images/all_frames/000004.jpg: 1088x1920 22 players, 3.1ms
Speed: 2.6ms preprocess, 3.1ms inference, 0.6ms postprocess 

In [26]:
boxann = sv.BoxAnnotator(
    thickness=1,
    text_thickness=1,
    text_scale=0.5,
    
)

for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_thickness=1):
        frame = result.orig_img
        
        detections = sv.Detections.from_ultralytics(result)
        

        frame = boxann.annotate(scene=frame, detections=detections)
        cv2.imshow('yolov8', frame)
        cv2.waitKey(1)  # Venter en kort tid så bildet kan oppdateres

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'line_width' instead.

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 22 players, 4.5ms


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
for frame in frame_paths:
    results = model.track(frame, persist=True)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    for box, id in zip(boxes, ids):
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Id {id}",
            (box[0], box[1]),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
        )
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break


image 1/1 /work/mbergst/project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 2 players, 3.2ms
Speed: 21.6ms preprocess, 3.2ms inference, 0.6ms postprocess per image at shape (1, 3, 1088, 1920)


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
